# Support Vector Machines (SVMs)

## Lecture plan

- SVMs: concepts and terminology.
  - Margins, hyperplanes, and support vectors.
  - Linear vs. non-linear separability.
- Using SVMs in `sklearn`. 

### Load libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # makes figs nicer!

## What is an SVM?

> A **Support Vector Machine (SVM)** is a generalization of a *maximal margin classifier*, in which the goal is to draw a **hyperplane** through some feature space separating your data into different classes.

- An SVM is a type of *classifier*.
- In this section, we'll discuss some key *concepts* that are important for understanding SVMs.
   - Hyperplanes.
   - Maximal margin classifiers.
   - Support vector classifiers.
   - Non-linear separability.

### What is a *hyperplane*?

> In some $p$-dimensional space, a **hyperplane** is a flat subspace of dimension $p-1$.

- In a $2D$ space, a hyperplane is just a line.
- In a $3D$ space, a hyperplane is a two-dimensional plane.
- And so on...though it's very hard to visualize anything past that!

In a $2D$ space, a hyperplane is defined by the following linear equation:

$$\beta_0 + \beta_1X_1 + \beta_2X_2 = 0$$

For any point $(X_1, X_2)$ that satisfies this equation (i.e., $=0$), that point **lies on the hyperplane**.

### Why do we care about hyperplanes?

- Many points *won't* lie on the hyperplane.
- That means we can treat the hyperplane as a **boundary**.
  - Classify points on *one* side as class $K_1$.
  - Classify points on the *other* side as class $K_2$.
- In the case of a **maximal margin classifier**, we want our hyperplane to maximally separate our classes.

<img src="img/hyperplane.png" width="300" alt="Hyperplane example">


### Finding the best hyperplane

- Given two **categories** of observations, there are a number of hyperplanes we might draw to separate them.
- Our goal is to fit the hyperplane with the largest **margin** $M$, i.e., the distance between the hyperplane and all of the closest points.

<img src="img/multiple_hyperplanes.png" width="500" alt="Multiple hyperplanes">


#### Margins and support vectors

- The **margin** is the distance from the solid line to either of the dashed lines (representing the closest points).
- The points closest to the hyperplane are called **support vectors**.
   - They "support" the hyperplane in that: *if they moved, the hyperplane would also change*.
   - The hyperplane really depends only on these support vectors!

<img src="img/support_vectors.png" width="300" alt="Support vectors">


#### A perfect boundary doesn't always exist...

- Often, there *doesn't exist* a hyperplane that perfectly separates our classes.
- In these cases, we need to relax some of our assumptions.
- We'll need to allow for **soft margins**, i.e., some overlap between our classes.
- This generalization is called a **support vector classifier**.

<img src="img/soft_margin.png" width="300" alt="Soft margin">


### Support vector classifiers, explained

> A **support vector classifier** (or *soft margin classifier*) classifies a test observation based on which side of the hyperplane it lies.

The SVC is a solution to the following constraints:

- Maximize $M$, i.e., **margin**.
- Subject to $\sum_{j=1}^p\beta_j^2=1$, i.e., sum of squared coefficients equals $1$.
- Allowing for some **slack variables** $\epsilon_1, ..., \epsilon_n$ that can end up on the wrong side of the classifier.
   - A tuning parameter $C$ determines how many of these were allow (i.e., our **budget** for misclassifications).


#### Check-in: the role of $C$

The parameter $C$ is a *budget* for how many misclassifications we allow when fitting our hyperplane. How might this relate to the bias-variance trade-off?


#### $C$ and the bias-variance trade-off

- If $C = 0$, we have no budget, i.e., it's a simple *maximal margin classifier*.
  - Ends up with classifier **highly fit to training data**.
  - High variance, low bias.
- A large $C$ means a more tolerant fit.
  - Low variance, higher bias.

Recall that only the **support vectors** affect the margin/hyperplane. $C$ essentially controls how many support vectors we care about.

#### Illustrating the role of $C$

- A large $C$ (top left) results in larger margin.
- A small $C$ (bottom right) results in smaller margin.
- The size of the margin determines how many points are considered in drawing the hyperplane, i.e., the **support vectors**!

<img src="img/margins_budget.png" width="300" alt="Tuning parameter">


### Support vector classifiers: an interim summary

- Our goal: classify data into categories.
- The *how*: draw a **hyperplane** separating the feature-space into classes.
  - The **margin** of the hyperplane is the distance to the nearest points.
  - Technically, only the nearest points (the **support vectors**) affect the maximal margin classifier.
- A support vector classifier allows for **soft margins**, which decrease variance and make more robust models.

So far, however, we've only considered classes with a *linear* decision boundary...

### The problem of non-linear separability

> [Linear separability](https://en.wikipedia.org/wiki/Linear_separability) means that two sets of points can be separated drawing a line (or flat hyperplane). **Non-linear** separability is when this is not the case!

In some cases, it's impossible to separate our categories with a linear boundary.

<img src="img/non_linear_separability.png" width="500" alt="The problem of non-linear separability">

#### Analogy: polynomial regression regression

- As discussed in CSS 2, [**polynomial regression**](https://ucsd-css2.github.io/ucsd-css2-website/lectures/16-nonlinear-regression.html) is useful for learning non-linear relationships between $X$ and $Y$.

$$Y = \beta_0 + \beta_1X_1 + \beta_2X_1^2 + ... \beta_pX_1^p$$

- With polynomial regression, we enter the same feature ($X_1$) multiple times in the model, but **transform** the feature into higher-order polynomials.
   - We then **fit the coefficients** using ordinary least squares.
- With an SVM, we can apply the same logic to our **decision boundary**.


#### Learning a non-linear decision boundary

- As in polynomial regression, we can *enlarge our feature space* using polynomial transformations (i.e., quadratic, cubic, etc.).
- In this enlarged feature space, we can fit a linear decision boundary.
  - However, in *original* feature space, this decision boundary is non-linear!
- The key question is how to enlarge the feature space.
- This is called the **kernel trick**.

### The "kernel trick"

> The **kernel trick** maps a lower-dimensional feature-space into a higher-dimensional space, in which a linear decision boundary can be drawn.

- E.g., map from $1D$ space to $3D$ space using some **transformation**.
   - Analogy: transforming a single predictor $X_1$ using cubic regression.
- People tend to use one of three kernels:
   - **Linear kernel**: linear projection, cannot handle non-linear decision boundaries.
   - **Polynomial kernel**: non-linear projection, analogous to polynomial regression.
   - **Radial basis function kernel**: another kind of non-linear projection, technically infinite feature space.
- Tends to be more computationally efficient than just transforming the features themselves.
   - Involves computing *inner product* of pairs of data points.

### The kernel trick works!

- A *linear kernel projection* (left) will fail to separate non-linearly separable data.
- But a **non-linear kernel projection** (right: radial basis function) will more often succeed.

<img src="img/kernel_trick.png" width="500" alt="The non-linear kernel trick works">

### Summing up: where do we stand?

- We've discussed a few key concepts:
   - **Hyperplane**: draw this to separate your classes.
   - **Margin**: distance from hyperplane to the nearest points.
   - **Support vectors**: the points that actually affect the position of the hyperplane.
   - **Non-linear separability**: when two clusters of points can't be separated with a linear decision boundary.
   - **Kernel trick**: a way to *project* data into a higher-dimensional space; non-linear kernel tricks allow you to solve non-linearly separable problems.
- Putting it altogether...

> A **support vector machine (SVM)** is a type of classifier that uses kernel tricks to draw hyperplanes that solve either linearly or non-linearly separable problems.

## SVMs in practice

- Now that we've discussed the conceptual background, we can review the `sklearn` implementation.
- [`sklearn.svm.SVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) is the relevant class.


### The `sklearn` implementation

- The [`SVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) class can be used to fit linear or non-linear kernels.
- Like other `sklearn` models, uses `.fit(X, y)` and `.predict(X)` syntax.
- Relevant parameters:
   - `C`: regularization parameter, determines *tolerance* for margin errors.
      - In `sklearn` implementation, `C` is *inversely proportional* to degree of regularization.
      - Larger value = less regularization.
   - `kernel`: e.g., *linear* vs. *poly* vs. *rbf*.

In [1]:
from sklearn.svm import SVC

### A sample dataset

- Let's return to our heart dataset.

In [4]:
df_heart = pd.read_csv("data/classification/heart.csv")
df_heart.head(3)

,Unnamed: 0,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,1,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
1,2,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
2,3,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes


In [20]:
### Relevant features
X = df_heart[['MaxHR', 'Chol', 'Age']]
y = df_heart['AHD'].values

#### Using `train_test_split`

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

#### Creating our `SVC` classifiers

In [23]:
### Create different models
linear_svc = SVC(kernel='linear')
radial_svc = SVC(kernel = 'rbf')
polynomial_svc = SVC(kernel = 'poly')

In [24]:
### Fit them
linear_svc.fit(X_train, y_train)
radial_svc.fit(X_train, y_train)
polynomial_svc.fit(X_train, y_train)

SVC(kernel='poly')

#### Evaluating our models

In [25]:
from sklearn.metrics import accuracy_score

In [26]:
y_pred_linear = linear_svc.predict(X_test)
y_pred_radial = radial_svc.predict(X_test)
y_pred_polynomial = polynomial_svc.predict(X_test)

In [28]:
### In this case, radial is best
print(accuracy_score(y_test, y_pred_linear))
print(accuracy_score(y_test, y_pred_radial))
print(accuracy_score(y_test, y_pred_polynomial))

0.64
0.66
0.64


### Which kernel to use?

- In practice, which **kernel trick** to use depends on your data.
   - Strategy 1: look at your data, try to estimate if classifier should be non-linear.
   - Strategy 2: empirical——fit multiple models and see which is best!
- The same applies to $C$.
   - Like with $\lambda$, can be **tuned** using cross-validation.

## Lecture wrap-up

- Support vector machines are a kind of **classification method**.
- Use linear or non-linear **kernel tricks** to project data to some higher-dimensional space, in which they draw a **hyperplane** separating the classes.
- Use `sklearn.svm.SVC` for Python implementation.